# Psi4 Notebook for educational purposes


This notebook is used to compare the results of Psi4 with the results with my own implementation of the Hartree-Fock method. It helps me to check if my implementation is correct. 

Connect to pc with `Psi4` installation and host jupyter server.

`ssh -L 8080:localhost:8080 username@pcname`

`jupyter notebook --no-browser --port=8080`

In [59]:
# cd /loctmp/dam63759/psi4_work

In [60]:
ls

h2o_test.out*          Notes.md        Psi4_compare.ipynb  water_HF.xyz*
h2o_testV2.out*        output.dat      psi4_setup_h2.py*
ipython_h2o_setup.py*  psi.1439.clean  water_HF_std.xyz*


My setup, that is run:

In [61]:
cat psi4_setup_h2.py

import psi4
import numpy as np 

np.set_printoptions(precision=5, linewidth=100, suppress=True)
psi4.set_memory("4 GB")
psi4.core.set_output_file('output.dat', False)
numpy_memory = 4

mol = psi4.geometry("""
0 1
H 0.0 0.0 0.0
H 0.0 0.0 1.4
units bohr
""")

psi4.set_options({"save_jk": "true", "guess": "CORE", "basis": "sto-3g", "DIIS": "false", "SCF_INITIAL_ACCELERATOR": "None", "CFOUR_SCF_MAXCYC": "500", "e_convergence": 1e-8})

wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())

S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

no_basis_funcs: int = S.shape[0]



In [62]:
%run psi4_setup_h2.py

In [63]:
import numpy as np 
import psi4
import scipy as sp 

### This is with STO-3G basis set 

In [64]:
V

array([[-1.88044, -1.19483],
       [-1.19483, -1.88044]])

In [65]:
S

array([[1.     , 0.65932],
       [0.65932, 1.     ]])

### This is with 6-311G basis set 

In [66]:
psi4.set_options({"save_jk": "true", "guess": "CORE", "basis": "6-311G", "DIIS": "false", "SCF_INITIAL_ACCELERATOR": "None", "MAXITER": 250, "CFOUR_SCF_MAXCYC": "500", "e_convergence": 1e-8, "CFOUR_CONTRACTION": "SEGMENTED", "SCF__PRINT_BASIS": "true"})

In [67]:
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())
S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V_ne = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

In [68]:
S

array([[1.     , 0.70638, 0.37366, 0.26948, 0.42526, 0.31026],
       [0.70638, 1.     , 0.78896, 0.42526, 0.72664, 0.67697],
       [0.37366, 0.78896, 1.     , 0.31026, 0.67697, 0.90422],
       [0.26948, 0.42526, 0.31026, 1.     , 0.70638, 0.37366],
       [0.42526, 0.72664, 0.67697, 0.70638, 1.     , 0.78896],
       [0.31026, 0.67697, 0.90422, 0.37366, 0.78896, 1.     ]])

In [69]:
T

array([[2.30435, 0.54912, 0.10635, 0.06934, 0.21812, 0.07735],
       [0.54912, 0.48876, 0.18488, 0.21812, 0.27955, 0.14245],
       [0.10635, 0.18488, 0.15411, 0.07735, 0.14245, 0.13   ],
       [0.06934, 0.21812, 0.07735, 2.30435, 0.54912, 0.10635],
       [0.21812, 0.27955, 0.14245, 0.54912, 0.48876, 0.18488],
       [0.07735, 0.14245, 0.13   , 0.10635, 0.18488, 0.15411]])

In [70]:
V_ne

array([[-2.82405, -1.55802, -0.77545, -0.7038 , -0.97076, -0.65609],
       [-1.55802, -1.54664, -1.03651, -0.97076, -1.19542, -0.92183],
       [-0.77545, -1.03651, -0.96188, -0.65609, -0.92183, -0.89488],
       [-0.7038 , -0.97076, -0.65609, -2.82405, -1.55802, -0.77545],
       [-0.97076, -1.19542, -0.92183, -1.55802, -1.54664, -1.03651],
       [-0.65609, -0.92183, -0.89488, -0.77545, -1.03651, -0.96188]])

In [71]:
ERI

array([[[[1.39303, 0.82297, 0.41126, 0.29007, 0.48009, 0.34014],
         [0.82297, 0.8229 , 0.54368, 0.33807, 0.54967, 0.46004],
         [0.41126, 0.54368, 0.49402, 0.22002, 0.40827, 0.4331 ],
         [0.29007, 0.33807, 0.22002, 0.6992 , 0.48072, 0.25081],
         [0.48009, 0.54967, 0.40827, 0.48072, 0.60763, 0.43572],
         [0.34014, 0.46004, 0.4331 , 0.25081, 0.43572, 0.43853]],

        [[0.82297, 0.5097 , 0.25776, 0.18349, 0.29937, 0.21335],
         [0.5097 , 0.54303, 0.36577, 0.22615, 0.3665 , 0.3099 ],
         [0.25776, 0.36577, 0.34012, 0.14922, 0.27791, 0.29863],
         [0.18349, 0.22615, 0.14922, 0.48072, 0.3286 , 0.17126],
         [0.29937, 0.3665 , 0.27791, 0.3286 , 0.41477, 0.29867],
         [0.21335, 0.3099 , 0.29863, 0.17126, 0.29867, 0.30363]],

        [[0.41126, 0.25776, 0.13079, 0.0933 , 0.15168, 0.10829],
         [0.25776, 0.27987, 0.1898 , 0.11698, 0.18956, 0.16089],
         [0.13079, 0.1898 , 0.17804, 0.07757, 0.14483, 0.15641],
         [0.0933 , 0.

In [72]:
S

array([[1.     , 0.70638, 0.37366, 0.26948, 0.42526, 0.31026],
       [0.70638, 1.     , 0.78896, 0.42526, 0.72664, 0.67697],
       [0.37366, 0.78896, 1.     , 0.31026, 0.67697, 0.90422],
       [0.26948, 0.42526, 0.31026, 1.     , 0.70638, 0.37366],
       [0.42526, 0.72664, 0.67697, 0.70638, 1.     , 0.78896],
       [0.31026, 0.67697, 0.90422, 0.37366, 0.78896, 1.     ]])

In [73]:
from scipy.linalg import sqrtm
S_sqrt = sqrtm(S)
S_sqrt_inv = np.linalg.inv(S_sqrt)

In [74]:
S_sqrt_inv

array([[ 1.37587, -0.74986,  0.21741, -0.02169, -0.00035, -0.03645],
       [-0.74986,  2.40629, -1.16249, -0.00035, -0.66585,  0.40979],
       [ 0.21741, -1.16249,  2.83156, -0.03645,  0.40979, -1.672  ],
       [-0.02169, -0.00035, -0.03645,  1.37587, -0.74986,  0.21741],
       [-0.00035, -0.66585,  0.40979, -0.74986,  2.40629, -1.16249],
       [-0.03645,  0.40979, -1.672  ,  0.21741, -1.16249,  2.83156]])

In [75]:
H_core = T + V_ne
H_core

array([[-0.5197 , -1.0089 , -0.66911, -0.63447, -0.75264, -0.57875],
       [-1.0089 , -1.05788, -0.85163, -0.75264, -0.91588, -0.77939],
       [-0.66911, -0.85163, -0.80777, -0.57875, -0.77939, -0.76488],
       [-0.63447, -0.75264, -0.57875, -0.5197 , -1.0089 , -0.66911],
       [-0.75264, -0.91588, -0.77939, -1.0089 , -1.05788, -0.85163],
       [-0.57875, -0.77939, -0.76488, -0.66911, -0.85163, -0.80777]])

In [76]:
F_0_pr = np.dot(S_sqrt_inv, np.dot(H_core, S_sqrt_inv))
F_0_pr

array([[ 0.3867 , -1.05935,  0.04115, -0.25515, -0.11328, -0.09317],
       [-1.05935,  0.11404, -0.3829 , -0.11328, -0.16925, -0.01183],
       [ 0.04115, -0.3829 , -0.32993, -0.09317, -0.01183, -0.13737],
       [-0.25515, -0.11328, -0.09317,  0.3867 , -1.05935,  0.04115],
       [-0.11328, -0.16925, -0.01183, -1.05935,  0.11404, -0.3829 ],
       [-0.09317, -0.01183, -0.13737,  0.04115, -0.3829 , -0.32993]])

In [77]:
from scipy.linalg import eigh
orb_energy_list, C_MO_basis = eigh(F_0_pr)
C_AO_basis = np.dot(S_sqrt_inv, C_MO_basis)
C_AO_basis

array([[-0.25139,  0.19186, -0.12421, -0.15775, -1.06586,  1.10815],
       [-0.34193,  0.77236, -0.5325 , -1.27058,  1.29568, -2.00624],
       [-0.01694,  0.82144,  0.80552,  2.92229, -0.56816,  1.48145],
       [-0.25139, -0.19186, -0.12421,  0.15775, -1.06586, -1.10815],
       [-0.34193, -0.77236, -0.5325 ,  1.27058,  1.29568,  2.00624],
       [-0.01694, -0.82144,  0.80552, -2.92229, -0.56816, -1.48145]])

In [78]:
no_basis_funcs = C_AO_basis.shape[0]
D_matr = np.zeros((no_basis_funcs, no_basis_funcs))
for mu in range(no_basis_funcs):
    for nu in range(no_basis_funcs):
        for i in range(3):
            D_matr[mu, nu] += C_AO_basis[mu, i] * C_AO_basis[nu, i]

D_matr

array([[ 0.11543,  0.30028,  0.06181,  0.04182,  0.00392, -0.25339],
       [ 0.30028,  0.99701,  0.2113 ,  0.00392, -0.19606, -1.0576 ],
       [ 0.06181,  0.2113 ,  1.32391, -0.25339, -1.0576 , -0.02563],
       [ 0.04182,  0.00392, -0.25339,  0.11543,  0.30028,  0.06181],
       [ 0.00392, -0.19606, -1.0576 ,  0.30028,  0.99701,  0.2113 ],
       [-0.25339, -1.0576 , -0.02563,  0.06181,  0.2113 ,  1.32391]])

In [79]:
energy = psi4.energy("SCF", molecule=mol)

In [80]:
energy

-1.1279785387000567

### Writing a function to compare the results -> help identify the problem

In [81]:
scf_mrjd_rs_T = np.array(
[[2.304346, 0.549119, 0.106346, -3.160976, 0.322166, 0.088218],
 [0.549119, 0.488760, 0.184881, -9.113568, 0.279547, 0.157027],
 [0.106346, 0.184881, 0.154112, -6.820726, -0.004220, 0.129996],
 [-3.160976, 0.322166, 0.088218, 2.304346, 0.549119, 0.106346],
 [-9.113568, 0.279547, 0.157027, 0.549119, 0.488760, 0.184881],
 [-6.820726, -0.004220, 0.129996, 0.106346, 0.184881, 0.154112]]
)

In [82]:
T

array([[2.30435, 0.54912, 0.10635, 0.06934, 0.21812, 0.07735],
       [0.54912, 0.48876, 0.18488, 0.21812, 0.27955, 0.14245],
       [0.10635, 0.18488, 0.15411, 0.07735, 0.14245, 0.13   ],
       [0.06934, 0.21812, 0.07735, 2.30435, 0.54912, 0.10635],
       [0.21812, 0.27955, 0.14245, 0.54912, 0.48876, 0.18488],
       [0.07735, 0.14245, 0.13   , 0.10635, 0.18488, 0.15411]])

In [83]:
is_close_T = np.isclose(scf_mrjd_rs_T, T, atol=0.0005)

In [84]:
for i in range(len(is_close_T)):
    for j in range(len(is_close_T[i])):
        if is_close_T[i][j] == False:
            print(f"{i},{j}")

0,3
0,4
0,5
1,3
1,5
2,3
2,4
3,0
3,1
3,2
4,0
4,2
5,0
5,1


In [85]:
scf_mrjd_rs_S = np.array(
[[0.999999, 0.706377, 0.373664, 0.269482, 0.425257, 0.310263],
 [0.706377, 1.000000, 0.788963, 0.425257, 0.726641, 0.676966],
 [0.373664, 0.788963, 1.000000, 0.310263, 0.676966, 0.904217],
 [0.269482, 0.425257, 0.310263, 0.999999, 0.706377, 0.373664],
 [0.425257, 0.726641, 0.676966, 0.706377, 1.000000, 0.788963],
 [0.310263, 0.676966, 0.904217, 0.373664, 0.788963, 1.000000]]
)

In [86]:
all_close_S = np.allclose(scf_mrjd_rs_S, S, atol=0.0005)
all_close_S

True

Test für $\text{BH}_3$

In [87]:
mol_BH3 = psi4.geometry(
    """
B          1.49587       -0.00587        0.00000
H          2.67699       -0.00587        0.00000
H          0.90531        0.59412       -0.82842
H          0.90531       -0.60586        0.82842
"""
)

psi4.set_options(
    {
        "save_jk": "true",
        "guess": "CORE",
        "basis": "sto-3g",
        "DIIS": "false",
        "SCF_INITIAL_ACCELERATOR": "None",
        "MAXITER": 250,
        "CFOUR_SCF_MAXCYC": "500",
        "e_convergence": 1e-8,
        "CFOUR_CONTRACTION": "SEGMENTED",
        "SCF__PRINT_BASIS": "true",
    }
)
wfn = psi4.core.Wavefunction.build(mol_BH3, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())

S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

no_basis_funcs: int = S.shape[0]


In [88]:
S

array([[ 1.     ,  0.26933, -0.     , -0.     ,  0.     ,  0.06786,  0.06786,  0.06786],
       [ 0.26933,  1.     , -0.     ,  0.     ,  0.     ,  0.48046,  0.48046,  0.48046],
       [-0.     , -0.     ,  1.     ,  0.     ,  0.     ,  0.49115, -0.24558, -0.24558],
       [-0.     ,  0.     ,  0.     ,  1.     ,  0.     , -0.     ,  0.     , -0.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  1.     ,  0.     , -0.42535,  0.42535],
       [ 0.06786,  0.48046,  0.49115, -0.     ,  0.     ,  1.     ,  0.11054,  0.11054],
       [ 0.06786,  0.48046, -0.24558,  0.     , -0.42535,  0.11054,  1.     ,  0.11054],
       [ 0.06786,  0.48046, -0.24558, -0.     ,  0.42535,  0.11054,  0.11054,  1.     ]])

In [89]:
no_basis_funcs

8

Test mit $\text{CH}_4$

In [90]:
mol_CH4 = psi4.geometry(
    """
C          1.49587       -0.00587        0.00000
H          2.56587       -0.00587        0.00000
H          1.13921        0.68968        0.73069
H          1.13921       -0.98644        0.23702
H          1.13920        0.27915       -0.96770
"""
)
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())

S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

no_basis_funcs: int = S.shape[0]


In [91]:
mol_GaH3 = psi4.geometry(
    """
Ga          1.49587       -0.00587        0.00000
H          2.67699       -0.00587        0.00000
H          0.90531        0.59412       -0.82842
H          0.90531       -0.60586        0.82842
"""
)

psi4.set_options(
    {
        "save_jk": "true",
        "guess": "CORE",
        "basis": "sto-3g",
        "DIIS": "false",
        "SCF_INITIAL_ACCELERATOR": "None",
        "MAXITER": 250,
        "CFOUR_SCF_MAXCYC": "500",
        "e_convergence": 1e-8,
        "CFOUR_CONTRACTION": "SEGMENTED",
        "SCF__PRINT_BASIS": "true",
    }
)
wfn = psi4.core.Wavefunction.build(mol_GaH3, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())

S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

no_basis_funcs: int = S.shape[0]


In [92]:
S

array([[ 1.     ,  0.39971,  0.     ,  0.     ,  0.     ,  0.02626, -0.     , -0.     ,  0.     ,
        -0.00035, -0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
         0.00394,  0.00394,  0.00394],
       [ 0.39971,  1.     , -0.     , -0.     ,  0.     ,  0.31357,  0.     , -0.     ,  0.     ,
         0.00314,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
         0.02457,  0.02457,  0.02457],
       [ 0.     , -0.     ,  1.     ,  0.     ,  0.     ,  0.     ,  0.32652,  0.     ,  0.     ,
         0.     ,  0.01028,  0.     ,  0.     ,  0.     , -0.     ,  0.     ,  0.     ,  0.     ,
         0.00551, -0.00276, -0.00276],
       [ 0.     , -0.     ,  0.     ,  1.     ,  0.     ,  0.     ,  0.     ,  0.32652,  0.     ,
        -0.     ,  0.     ,  0.01028,  0.     ,  0.     ,  0.     ,  0.     , -0.     ,  0.     ,
         0.     ,  0.     , -0.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  1.     ,  0

In [93]:
no_basis_funcs

21

In [94]:
mol_H2O = psi4.geometry(
    """
units bohr
O   0.000000000000  -0.143225816552   0.000000000000
H   1.638036840407   1.136548822547  -0.000000000000
H  -1.638036840407   1.136548822547  -0.000000000001
"""
)

psi4.set_options(
    {
        "save_jk": "true",
        "guess": "CORE",
        "basis": "sto-3g",
        "DIIS": "false",
        "SCF_INITIAL_ACCELERATOR": "None",
        "MAXITER": 250,
        "CFOUR_SCF_MAXCYC": "500",
        "e_convergence": 1e-8,
        # "CFOUR_CONTRACTION": "SEGMENTED",
        "SCF__PRINT_BASIS": "true",
    }
)
wfn = psi4.core.Wavefunction.build(mol_H2O, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())

S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

no_basis_funcs: int = S.shape[0]
E_tot = psi4.energy("SCF", molecule=mol_H2O)


In [95]:
S

array([[ 1.     ,  0.2367 , -0.     ,  0.     ,  0.     ,  0.03841,  0.03841],
       [ 0.2367 ,  1.     , -0.     ,  0.     ,  0.     ,  0.38614,  0.38614],
       [-0.     , -0.     ,  1.     ,  0.     , -0.     ,  0.20973,  0.20973],
       [ 0.     ,  0.     ,  0.     ,  1.     ,  0.     ,  0.26844, -0.26844],
       [ 0.     ,  0.     , -0.     ,  0.     ,  1.     ,  0.     , -0.     ],
       [ 0.03841,  0.38614,  0.20973,  0.26844,  0.     ,  1.     ,  0.18176],
       [ 0.03841,  0.38614,  0.20973, -0.26844, -0.     ,  0.18176,  1.     ]])

In [96]:
T

array([[29.0032 , -0.16801,  0.     ,  0.     , -0.     , -0.00842, -0.00842],
       [-0.16801,  0.80813, -0.     ,  0.     ,  0.     ,  0.07052,  0.07052],
       [ 0.     , -0.     ,  2.52873,  0.     , -0.     ,  0.11492,  0.11492],
       [ 0.     ,  0.     ,  0.     ,  2.52873,  0.     ,  0.14709, -0.14709],
       [-0.     ,  0.     , -0.     ,  0.     ,  2.52873,  0.     , -0.     ],
       [-0.00842,  0.07052,  0.11492,  0.14709,  0.     ,  0.76003, -0.00398],
       [-0.00842,  0.07052,  0.11492, -0.14709, -0.     , -0.00398,  0.76003]])

In [97]:
V

array([[-61.5806 ,  -7.41082,  -0.01447,   0.     ,   0.     ,  -1.23169,  -1.23169],
       [ -7.41082, -10.00907,  -0.17689,   0.     ,   0.     ,  -2.97723,  -2.97723],
       [ -0.01447,  -0.17689,  -9.94404,   0.     ,   0.     ,  -1.47179,  -1.47179],
       [  0.     ,   0.     ,   0.     ,  -9.98755,  -0.     ,  -1.82224,   1.82224],
       [  0.     ,   0.     ,   0.     ,  -0.     ,  -9.87588,  -0.     ,   0.     ],
       [ -1.23169,  -2.97723,  -1.47179,  -1.82224,  -0.     ,  -5.3002 ,  -1.06717],
       [ -1.23169,  -2.97723,  -1.47179,   1.82224,   0.     ,  -1.06717,  -5.3002 ]])

In [116]:
ERI[0,0,5,2]

0.14339605057641877

In [99]:
E_tot

-74.94217612606148

In [100]:
H_core = T + V

In [101]:
H_core

array([[-32.5774 ,  -7.57883,  -0.01447,   0.     ,  -0.     ,  -1.2401 ,  -1.2401 ],
       [ -7.57883,  -9.20094,  -0.17689,   0.     ,   0.     ,  -2.90671,  -2.90671],
       [ -0.01447,  -0.17689,  -7.41531,   0.     ,  -0.     ,  -1.35687,  -1.35687],
       [  0.     ,   0.     ,   0.     ,  -7.45882,  -0.     ,  -1.67515,   1.67515],
       [ -0.     ,   0.     ,  -0.     ,  -0.     ,  -7.34714,  -0.     ,   0.     ],
       [ -1.2401 ,  -2.90671,  -1.35687,  -1.67515,  -0.     ,  -4.54017,  -1.07115],
       [ -1.2401 ,  -2.90671,  -1.35687,   1.67515,   0.     ,  -1.07115,  -4.54017]])

In [102]:
S_sqrt_inv = sp.linalg.inv(sp.linalg.sqrtm(S))
S_sqrt_inv

array([[ 1.02363, -0.13685, -0.00749, -0.     ,  0.     ,  0.01903,  0.01903],
       [-0.13685,  1.15786,  0.07216, -0.     , -0.     , -0.22233, -0.22233],
       [-0.00749,  0.07216,  1.03831,  0.     , -0.     , -0.11846, -0.11846],
       [-0.     , -0.     ,  0.     ,  1.07331,  0.     , -0.17576,  0.17576],
       [ 0.     , -0.     , -0.     ,  0.     ,  1.     , -0.     ,  0.     ],
       [ 0.01903, -0.22233, -0.11846, -0.17576, -0.     ,  1.12972, -0.0626 ],
       [ 0.01903, -0.22233, -0.11846,  0.17576,  0.     , -0.0626 ,  1.12972]])

In [103]:
F_0_pr = S_sqrt_inv @ H_core @ S_sqrt_inv
F_0_pr

array([[-32.25459,  -2.79149,   0.00861,   0.     ,  -0.     ,  -0.1813 ,  -0.1813 ],
       [ -2.79149,  -8.23689,  -0.22829,   0.     ,  -0.     ,  -0.3858 ,  -0.3858 ],
       [  0.00861,  -0.22829,  -7.45703,  -0.     ,  -0.     ,  -0.11022,  -0.11022],
       [  0.     ,   0.     ,  -0.     ,  -7.54289,   0.     ,  -0.11321,   0.11321],
       [ -0.     ,  -0.     ,   0.     ,   0.     ,  -7.34714,  -0.     ,   0.     ],
       [ -0.1813 ,  -0.3858 ,  -0.11022,  -0.11321,  -0.     ,  -4.03295,  -0.04465],
       [ -0.1813 ,  -0.3858 ,  -0.11022,   0.11321,   0.     ,  -0.04465,  -4.03295]])

In [104]:
C_MO_basis = sp.linalg.eigh(F_0_pr)[1]
C_MO_basis

array([[ 0.9934 ,  0.10681,  0.     , -0.0417 ,  0.     ,  0.00366, -0.     ],
       [ 0.11418, -0.91397, -0.     ,  0.3677 , -0.     , -0.12815,  0.     ],
       [ 0.00077, -0.36854,  0.     , -0.92891,  0.     , -0.03624,  0.     ],
       [-0.     , -0.     ,  0.99899,  0.     , -0.     ,  0.     ,  0.04491],
       [ 0.     , -0.     ,  0.     ,  0.     ,  1.     ,  0.     ,  0.     ],
       [ 0.00787, -0.09337,  0.03175,  0.00971, -0.     ,  0.7008 , -0.70639],
       [ 0.00787, -0.09337, -0.03175,  0.00971,  0.     ,  0.7008 ,  0.70639]])

In [105]:
C_AO_basis = S_sqrt_inv @ C_MO_basis
C_AO_basis

array([[ 1.00154,  0.23362,  0.     , -0.08568, -0.     ,  0.04822, -0.     ],
       [-0.00719, -1.05794, -0.     ,  0.36011,  0.     , -0.46312,  0.     ],
       [-0.00027, -0.42728,  0.     , -0.93994,  0.     , -0.21294,  0.     ],
       [-0.     , -0.     ,  1.06107,  0.     , -0.     ,  0.     ,  0.29651],
       [ 0.     , -0.     ,  0.     ,  0.     ,  1.     ,  0.     ,  0.     ],
       [ 0.00182,  0.14925, -0.13772,  0.03786,  0.     ,  0.7807 , -0.85014],
       [ 0.00182,  0.14925,  0.13772,  0.03786, -0.     ,  0.7807 ,  0.85014]])

In [106]:
D_matr = C_AO_basis[:, :5] @ C_AO_basis[:, :5].T
D_matr

array([[ 1.06501, -0.28522, -0.01955, -0.     ,  0.     ,  0.03345,  0.03345],
       [-0.28522,  1.24897,  0.11356, -0.     , -0.     , -0.14428, -0.14428],
       [-0.01955,  0.11356,  1.06606,  0.     ,  0.     , -0.09936, -0.09936],
       [-0.     , -0.     ,  0.     ,  1.12587, -0.     , -0.14613,  0.14613],
       [ 0.     , -0.     ,  0.     , -0.     ,  1.     ,  0.     , -0.     ],
       [ 0.03345, -0.14428, -0.09936, -0.14613,  0.     ,  0.04268,  0.00475],
       [ 0.03345, -0.14428, -0.09936,  0.14613, -0.     ,  0.00475,  0.04268]])

In [107]:
E_scf = 2 * np.einsum("mn, mn", D_matr, H_core) 
E_scf

-125.84207743769926

In [108]:
F_matr = H_core + 2 * np.einsum("pqrs, rs", ERI, D_matr) - np.einsum("prqs, rs", ERI, D_matr)

In [109]:
F_matr

array([[-18.81327,  -4.87269,  -0.01153,   0.     ,  -0.     ,  -0.80673,  -0.80673],
       [ -4.87269,  -1.7909 ,  -0.18087,  -0.     ,   0.     ,  -0.57906,  -0.57906],
       [ -0.01153,  -0.18087,   0.23912,  -0.     ,  -0.     ,  -0.18287,  -0.18287],
       [  0.     ,  -0.     ,  -0.     ,   0.19396,   0.     ,  -0.17089,   0.17089],
       [ -0.     ,   0.     ,  -0.     ,   0.     ,   0.30911,  -0.     ,   0.     ],
       [ -0.80673,  -0.57906,  -0.18287,  -0.17089,  -0.     ,  -0.14503,  -0.18467],
       [ -0.80673,  -0.57906,  -0.18287,   0.17089,   0.     ,  -0.18467,  -0.14503]])

1. real step in SCF

In [110]:
F_matr_pr = S_sqrt_inv @ F_matr @ S_sqrt_inv
F_matr_pr

array([[-18.43852,  -2.6325 ,  -0.0219 ,   0.     ,  -0.     ,  -0.09766,  -0.09766],
       [ -2.6325 ,  -0.76061,  -0.0586 ,  -0.     ,   0.     ,  -0.3137 ,  -0.3137 ],
       [ -0.0219 ,  -0.0586 ,   0.32341,  -0.     ,  -0.     ,  -0.20082,  -0.20082],
       [  0.     ,  -0.     ,  -0.     ,   0.35484,   0.     ,  -0.27415,   0.27415],
       [ -0.     ,   0.     ,  -0.     ,   0.     ,   0.30911,  -0.     ,   0.     ],
       [ -0.09766,  -0.3137 ,  -0.20082,  -0.27415,  -0.     ,   0.14607,  -0.0655 ],
       [ -0.09766,  -0.3137 ,  -0.20082,   0.27415,   0.     ,  -0.0655 ,   0.14607]])

In [111]:
C_AO_basis = S_sqrt_inv @ sp.linalg.eigh(F_matr_pr)[1]
C_AO_basis

array([[ 0.99334,  0.22547, -0.     , -0.14885,  0.     , -0.0873 ,  0.     ],
       [ 0.02864, -0.80169,  0.     ,  0.72386, -0.     ,  0.54401, -0.     ],
       [ 0.00303, -0.17678, -0.     , -0.56053, -0.     ,  0.87519, -0.     ],
       [-0.     , -0.     , -0.49549,  0.     , -0.     , -0.     , -0.98401],
       [ 0.     ,  0.     , -0.     , -0.     ,  1.     ,  0.     , -0.     ],
       [-0.00546, -0.19819, -0.53564, -0.4455 ,  0.     , -0.62883,  0.67439],
       [-0.00546, -0.19819,  0.53564, -0.4455 , -0.     , -0.62883, -0.67439]])

In [112]:
D_matr_prev = D_matr.copy()

In [113]:
D_matr = C_AO_basis[:, :5] @ C_AO_basis[:, :5].T
D_matr

array([[ 1.05972, -0.26005,  0.04659, -0.     ,  0.     ,  0.0162 ,  0.0162 ],
       [-0.26005,  1.1675 , -0.26394,  0.     , -0.     , -0.16375, -0.16375],
       [ 0.04659, -0.26394,  0.34546, -0.     , -0.     ,  0.28474,  0.28474],
       [-0.     ,  0.     , -0.     ,  0.24551, -0.     ,  0.2654 , -0.2654 ],
       [ 0.     , -0.     , -0.     , -0.     ,  1.     ,  0.     , -0.     ],
       [ 0.0162 , -0.16375,  0.28474,  0.2654 ,  0.     ,  0.52469, -0.04912],
       [ 0.0162 , -0.16375,  0.28474, -0.2654 , -0.     , -0.04912,  0.52469]])

In [114]:
E_scf = np.einsum("mn, mn", D_matr, (H_core + F_matr))
E_scf 

-78.28658328473648